## Dataset Download

In [1]:
!gdown 1LQoyq_ZjHJih7hxJ2yJ_OrZhOIE_BZwB

Downloading...
From (original): https://drive.google.com/uc?id=1LQoyq_ZjHJih7hxJ2yJ_OrZhOIE_BZwB
From (redirected): https://drive.google.com/uc?id=1LQoyq_ZjHJih7hxJ2yJ_OrZhOIE_BZwB&confirm=t&uuid=8aec5ebd-370f-48d8-b4b8-9474e3dd613b
To: /kaggle/working/depression_dataset_complete.zip
100%|████████████████████████████████████████| 652M/652M [00:09<00:00, 68.4MB/s]


In [2]:
!unzip depression_dataset_complete.zip > /dev/null

In [14]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import CLIPImageProcessor, CLIPVisionModel
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
from tqdm import tqdm
from PIL import Image
import json
import math

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 16
LR = 2e-5
FUSION_DIM = 768
LABELS = ["Lack of Interest", "Feeling Down", "Eating Disorder",
          "Sleeping Disorder", "Low Self-Esteem", "Concentration Problem", "Self-Harm"]
LABEL_MAP = {label: i for i, label in enumerate(LABELS)}
NUM_CLASSES = len(LABELS)

## Custom Dataset

In [15]:
class ImageOnlyDepressionDataset(Dataset):
    def __init__(self, data, image_path, image_processor):
        self.data = data
        self.image_processor = image_processor
        self.img_path = image_path
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]

        image_path = os.path.join(self.img_path, sample["sample_id"] + ".jpeg")
        image = Image.open(image_path).convert("RGB")
        image_tensor = self.image_processor(image, return_tensors="pt")

        labels = torch.zeros(NUM_CLASSES)
        for category in sample["meme_depressive_categories"]:
            labels[LABEL_MAP[category]] = 1.0

        return {
            "image": image_tensor,
            "label": labels
        }

def custom_collate_fn(batch):
    labels = torch.stack([item['label'] for item in batch])

    images = {}
    for key in batch[0]['image'].keys():
        if isinstance(batch[0]['image'][key], torch.Tensor):
            images[key] = torch.stack([item['image'][key].squeeze(0) for item in batch])

    return {
        'image': images,
        'label': labels,
    }

## Model Definition

In [16]:
class ImageOnlyModel(nn.Module):
    def __init__(self, num_classes=7, fusion_dim=768):
        super(ImageOnlyModel, self).__init__()

        self.vision_encoder = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32")
        self.vision_dim = self.vision_encoder.config.hidden_size

        fusion_dim = 1024

        self.vision_projection = nn.Linear(self.vision_dim, fusion_dim)

        self.classifier = nn.Sequential(
            nn.Linear(fusion_dim, fusion_dim),
            nn.LayerNorm(fusion_dim),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(fusion_dim, num_classes)
        )

    def forward(self, image_features):
        vision_outputs = self.vision_encoder(**image_features)
        image_cls = vision_outputs.pooler_output

        vision_features = self.vision_projection(image_cls)

        logits = self.classifier(vision_features)

        return logits

## Training Functions

In [17]:
def train_image_only_model(model, train_data, val_data, img_path, epochs, model_save_name):
    image_processor = CLIPImageProcessor.from_pretrained("openai/clip-vit-base-patch32")

    train_dataset = ImageOnlyDepressionDataset(train_data, os.path.join(img_path, "train"), image_processor)
    val_dataset = ImageOnlyDepressionDataset(val_data, os.path.join(img_path, "val"), image_processor)

    print("Train Set Size:", len(train_dataset))
    print("Validation Set Size:", len(val_dataset))

    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        collate_fn=custom_collate_fn
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        collate_fn=custom_collate_fn
    )

    optimizer = optim.AdamW(model.parameters(), lr=LR)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=2, verbose=True
    )
    criterion = nn.BCEWithLogitsLoss()

    model = model.to(DEVICE)
    model = nn.DataParallel(model)

    best_f1 = 0
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")

        model.train()
        train_loss = 0
        all_train_preds, all_train_labels = [], []

        for batch in tqdm(train_loader, desc="Training"):
            labels = batch["label"].to(DEVICE)
            image_features = {k: v.to(DEVICE) for k, v in batch["image"].items()}

            optimizer.zero_grad()
            logits = model(image_features)
            loss = criterion(logits, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            train_loss += loss.item()
            with torch.no_grad():
                predictions = (torch.sigmoid(logits) > 0.5).float().cpu().numpy()
                all_train_preds.extend(predictions)
                all_train_labels.extend(labels.cpu().numpy())

        train_loss = train_loss / len(train_loader)
        train_metrics = compute_multilabel_metrics(np.array(all_train_labels), np.array(all_train_preds))

        print(f"Train Loss: {train_loss:.4f}")
        print(f"Train Macro-F1: {train_metrics['macro_f1']:.4f}, Weighted-F1: {train_metrics['weighted_f1']:.4f}")

        val_loss, val_metrics = evaluate_image_only_model(
            model, val_loader, criterion
        )

        print(f"Validation Loss: {val_loss:.4f}")
        print(f"Validation Macro-F1: {val_metrics['macro_f1']:.4f}, Weighted-F1: {val_metrics['weighted_f1']:.4f}")

        scheduler.step(val_metrics['macro_f1'])

        scheduler.step(val_metrics['macro_f1'])
        
        f1_hm = 2 * val_metrics["macro_f1"] * val_metrics["weighted_f1"] / (val_metrics["macro_f1"] + val_metrics["weighted_f1"])
        if f1_hm > best_f1:
            best_f1 = f1_hm
            torch.save(model.state_dict(), f"{model_save_name}_depression.pth")
            print("Best model saved!")
        
    return model

def compute_multilabel_metrics(y_true, y_pred):
    macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
    weighted_precision, weighted_recall, weighted_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
    
    per_class_precision, per_class_recall, per_class_f1, _ = precision_recall_fscore_support(y_true, y_pred, average=None)
    metrics = {
        'macro_precision': macro_precision,
        'macro_recall': macro_recall,
        'macro_f1': macro_f1,
        'weighted_precision': weighted_precision,
        'weighted_recall': weighted_recall,
        'weighted_f1': weighted_f1,
        'per_class_precision': per_class_precision,
        'per_class_recall': per_class_recall,
        'per_class_f1': per_class_f1
    }
    
    return metrics

def evaluate_image_only_model(model, loader, criterion):
    model.eval()
    val_loss = 0
    all_val_preds, all_val_labels = [], []
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluating"):
            labels = batch["label"].to(DEVICE)
            image_features = {k: v.to(DEVICE) for k, v in batch["image"].items()}
            
            logits = model(image_features)
            loss = criterion(logits, labels)
            
            val_loss += loss.item()
            predictions = (torch.sigmoid(logits) > 0.5).float().cpu().numpy()
            all_val_preds.extend(predictions)
            all_val_labels.extend(labels.cpu().numpy())
    
    val_loss = val_loss / len(loader)
    val_metrics = compute_multilabel_metrics(np.array(all_val_labels), np.array(all_val_preds))
    
    print("\nPer-class metrics:")
    for i, label in enumerate(LABELS):
        print(f"{label}: F1={val_metrics['per_class_f1'][i]:.4f}, "
              f"Precision={val_metrics['per_class_precision'][i]:.4f}, "
              f"Recall={val_metrics['per_class_recall'][i]:.4f}")
    
    return val_loss, val_metrics

## Model Training

In [19]:
train_data = json.load(open("depression_train_llava_dataset.json", "r"))
val_data = json.load(open("depression_val_llava_dataset.json", "r"))
test_data = json.load(open("depression_test_llava_dataset.json", "r"))

image_processor = CLIPImageProcessor.from_pretrained("openai/clip-vit-base-patch32")

img_path = "depressive_image"

image_processor = CLIPImageProcessor.from_pretrained("openai/clip-vit-base-patch32")

test_dataset = ImageOnlyDepressionDataset(test_data, os.path.join(img_path, "test"), image_processor)
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=custom_collate_fn
)

In [ ]:
model = ImageOnlyModel(
    num_classes=NUM_CLASSES,
    fusion_dim=FUSION_DIM
)

trained_model = train_image_only_model(
    model,
    train_data,
    val_data,
    img_path,
    epochs=30,
    model_save_name="only_image"
)

Train Set Size: 8722
Validation Set Size: 359

Epoch 1/30


Training: 100%|██████████| 546/546 [03:55<00:00,  2.31it/s]


Train Loss: 0.3221
Train Macro-F1: 0.3554, Weighted-F1: 0.3887


Evaluating: 100%|██████████| 23/23 [00:06<00:00,  3.35it/s]



Per-class metrics:
Lack of Interest: F1=0.2963, Precision=0.8889, Recall=0.1778
Feeling Down: F1=0.3857, Precision=0.6353, Recall=0.2769
Eating Disorder: F1=0.4681, Precision=0.4889, Recall=0.4490
Sleeping Disorder: F1=0.6842, Precision=0.8387, Recall=0.5778
Low Self-Esteem: F1=0.0000, Precision=0.0000, Recall=0.0000
Concentration Problem: F1=0.5085, Precision=0.8824, Recall=0.3571
Self-Harm: F1=0.1159, Precision=0.5000, Recall=0.0656
Validation Loss: 0.4472
Validation Macro-F1: 0.3512, Weighted-F1: 0.3270
Best model saved!


...


Epoch 30/30


Training: 100%|██████████| 546/546 [03:54<00:00,  2.32it/s]


Train Loss: 0.0033
Train Macro-F1: 0.9939, Weighted-F1: 0.9943


Evaluating: 100%|██████████| 23/23 [00:06<00:00,  3.32it/s]



Per-class metrics:
Lack of Interest: F1=0.2857, Precision=0.7273, Recall=0.1778
Feeling Down: F1=0.5246, Precision=0.7273, Recall=0.4103
Eating Disorder: F1=0.4793, Precision=0.4028, Recall=0.5918
Sleeping Disorder: F1=0.4792, Precision=0.4510, Recall=0.5111
Low Self-Esteem: F1=0.1600, Precision=0.5333, Recall=0.0941
Concentration Problem: F1=0.5316, Precision=0.5676, Recall=0.5000
Self-Harm: F1=0.3429, Precision=0.4091, Recall=0.2951
Validation Loss: 1.1460
Validation Macro-F1: 0.4005, Weighted-F1: 0.4158


## Inference

In [20]:
!gdown 14hFndYWBAr5sHKU1PTb1V0jCcnJmdxj6

Downloading...
From (original): https://drive.google.com/uc?id=14hFndYWBAr5sHKU1PTb1V0jCcnJmdxj6
From (redirected): https://drive.google.com/uc?id=14hFndYWBAr5sHKU1PTb1V0jCcnJmdxj6&confirm=t&uuid=7b981daa-ae0c-4ccc-899f-a0eadcbcc2d8
To: /kaggle/working/only_image_depression.pth
100%|████████████████████████████████████████| 357M/357M [00:08<00:00, 43.8MB/s]


In [21]:
def inference(test_data, model_path):
    model = ImageOnlyModel(
        num_classes=NUM_CLASSES,
        fusion_dim=FUSION_DIM
    )
        
    model = model.to(DEVICE)

    weights = torch.load(model_path, map_location=DEVICE, weights_only=True)
    weights_single = {k.replace("module.", ""): v for k, v in weights.items()}

    model.load_state_dict(weights_single)

    image_processor = CLIPImageProcessor.from_pretrained("openai/clip-vit-base-patch32")

    test_dataset = ImageOnlyDepressionDataset(test_data, os.path.join(img_path, "test"), image_processor)
    test_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        collate_fn=custom_collate_fn
    )

    loss, metrics = evaluate_image_only_model(
        model, test_loader, nn.BCEWithLogitsLoss()
    )

    print(f"Test Loss: {loss:.4f}")
    print(f"Test Macro-F1: {metrics['macro_f1']:.4f}, Weighted-F1: {metrics['weighted_f1']:.4f}")

In [23]:
inference(test_data, "only_image_depression.pth")

Evaluating: 100%|██████████| 33/33 [00:10<00:00,  3.14it/s]


Per-class metrics:
Lack of Interest: F1=0.2222, Precision=0.5263, Recall=0.1408
Feeling Down: F1=0.5492, Precision=0.6310, Recall=0.4862
Eating Disorder: F1=0.4113, Precision=0.5918, Recall=0.3152
Sleeping Disorder: F1=0.5037, Precision=0.6071, Recall=0.4304
Low Self-Esteem: F1=0.2152, Precision=0.3864, Recall=0.1491
Concentration Problem: F1=0.7009, Precision=0.8039, Recall=0.6212
Self-Harm: F1=0.3284, Precision=0.2750, Recall=0.4074
Test Loss: 0.7051
Test Macro-F1: 0.4187, Weighted-F1: 0.4307
